In [ ]:
import json

import dpx_control_hw as dch
import dpx_energy_calibration

## Settings

In [ ]:
MODEL_FILE = '../models/calibration_large.h5'
PARAMETERS_FILE = '../models/parameters.json'
CONFIG = '../../measure/config_38.conf'

## Connect Dosepix

In [ ]:
port = dch.find_port()
if port is None:
    port = '/dev/ttyACM0'

dpx = dch.Dosepix(
    port_name=port,
    config_fn=CONFIG,
    thl_calib_fn=None,
    params_fn=None,
    bin_edges_fn=None
)

## Energy Calibration

In [ ]:
dec = dpx_energy_calibration.DPXEnergyCalibration(
    dpx,
    MODEL_FILE,
    PARAMETERS_FILE
)

## Create interactive plots

In [ ]:
fig_hist, fig_params = dec.create_plots()

### Spectra
of large pixels of first column and average of all large pixels

In [ ]:
fig_hist

## Parameters
Change of the means of the parameters as a function of number of evaluations. Additionally, the average is shown. Its value is used to determine convergence of the calibration

In [ ]:
fig_params

## Start Calibration
Once the measurement starts, check plots above for current results!

In [ ]:
STOP_CONDITION = 0.01
STOP_CONDITION_RANGE = 10
STOP_TIME = 10

params, hist = dec.calibrate(
    frame_time=0.05,
    eval_after_frames=100,
    stop_condition=STOP_CONDITION,
    stop_condition_range=STOP_CONDITION_RANGE,
    stop_time=STOP_TIME,
    plot=True
)

## Store and show results

In [ ]:
with open('params.json', 'w') as f:
    json.dump( dec.reformat_params(params), f)

### Distribution of parameters

In [ ]:
dec.params_distribution(params)

### Distribution of thresholds

In [ ]:
dec.thl_distribution(params)

### Distribution of calibration curves

In [ ]:
dec.calib_curve_distribution(params)